In [106]:
import cv2 as cv

In [107]:
classic_path = '../train/classic'

In [108]:
img = cv.imread(classic_path+"/26.jpg")
img = cv.resize(img,(0,0),fx=0.22,fy=0.22)
img.shape

(887, 665, 3)

In [109]:
def imshow(img):
    cv.imshow('image',img)
    cv.waitKey()
    cv.destroyAllWindows()

In [110]:
sudoku_array = []
import matplotlib.pyplot as plt
import numpy as np
import imutils
for i in range(1,51):
    print(i)
    img = cv.imread(classic_path+"/"+str(i)+".jpg",0)
    img = cv.resize(img,(0,0),fx=0.22,fy=0.22)
    original = img.copy()
    img = cv.GaussianBlur(img,(15,15),0)

    thresh = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,5,2)
#     kernel = np.ones((1,4), np.uint8)
#     img_erosion = cv.erode(thresh, kernel, iterations=1)
#     kernel = np.ones((4,1), np.uint8)
#     img_erosion = cv.erode(thresh, kernel, iterations=1)
    contours,_ = cv.findContours(thresh,cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    mock_image = thresh.copy()
    saved = None
    #last_peri = 0
    for cnt in contours:
        x,y,w,h = cv.boundingRect(cnt)
        perimeter = cv.arcLength(cnt,True)
        epsilon = 0.1*cv.arcLength(cnt,True)
        approx = cv.approxPolyDP(cnt,epsilon,True)
        #if w > 420 and h > 420 and x > 5 and y  > 5:
        if w > 300 and w < 800 and h > 300 and h < 800:
            if saved is None:
                saved = (x,y,x+w,y+h)
            if saved[3]-saved[1] < h:
                saved = (x,y,x+w,y+h)
    cv.rectangle(original,(saved[0],saved[1]),(saved[2],saved[3]),(0,255,0),2)
            
    original = cv.adaptiveThreshold(original,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv.THRESH_BINARY,73,35)
    #imshow(original)
    if saved is None:
        sudoku_array.append(original)

    sudoku_array.append(original[saved[1]:saved[3],saved[0]:saved[2]])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [111]:
# imshow(sudoku_array[25])

In [112]:
img = cv.imread(classic_path+"/"+str(1)+".jpg",0)
img = cv.resize(img,(0,0),fx=0.22,fy=0.22)
thresh = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv.THRESH_BINARY,73,35)
contours,_ = cv.findContours(thresh, 1, 2)
mock_image = thresh.copy()
rect =  0
for cnt in contours:
    x,y,w,h = cv.boundingRect(cnt)

    if w > 500 and h > 500 and x !=0:
        #cv.rectangle(mock_image,(x,y),(x+w,y+h),(0,255,0),2)
        saved = (x,y,x+w,y+h)
print(saved)
#imshow(mock_image)

(20, 164, 634, 782)


In [113]:
mock_image = thresh.copy()
rect =  0
for cnt in contours:
    x,y,w,h = cv.boundingRect(cnt)
    epsilon = 0.1*cv.arcLength(cnt,True)
    approx = cv.approxPolyDP(cnt,epsilon,True)
    perimeter = cv.arcLength(cnt,True)
    #print(len(approx))
    #print(perimeter)
    if perimeter > 1000 and perimeter < 2000:
        #cv.rectangle(mock_image,(x,y),(x+w,y+h),(0,255,0),2)
        cv.drawContours(mock_image,[approx],0,(0,0,255),2)
        saved = (x,y,x+w,y+h)
print(saved)
#imshow(mock_image)

(40, 822, 570, 840)


In [114]:
import matplotlib.pyplot as plt
import math
from scipy import ndimage
from imutils.perspective import four_point_transform
rotated_sudoku = []
for idx,x in enumerate(sudoku_array):
    print(idx)
    edges = cv.Canny(x,50,100,apertureSize = 3)
    #x = cv.cvtColor(x,cv.COLOR_GRAY2BGR)
    lines = cv.HoughLines(edges,1,np.pi/180,200)
    #img = x.copy()
    #print(lines[0])
#     if lines is not None:
#         for i in range(0, len(lines)):
#             rho = lines[i][0][0]
#             theta = lines[i][0][1]
#             a = math.cos(theta)
#             b = math.sin(theta)
#             #print(a,b)
#             x0 = a * rho
#             y0 = b * rho
#             pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
#             pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
#             cv.line(img, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
    theta = lines[15][0][1]
    degree = theta*57
    first = x.shape[0]
    #print(x.shape)
    #print(degree)
    if degree >5 and degree <30:
        x = ndimage.rotate(x, degree)
    if degree < 89 and degree > 31:
        #print(90-degree)
        x = ndimage.rotate(x, -(89-degree))
    if degree > 92 and degree < 120:
        #print(abs(88-degree))
        x = ndimage.rotate(x, abs(89-degree))
    if degree > 121 and degree < 178:
        #print(360-200-degree)
        x = ndimage.rotate(x, -(180-degree))
    #x = ndimage.rotate(x, 45)
    diff = x.shape[0] - first
    if diff > 30:
        diff = diff//2
        x = x[diff+22:-(diff+22),diff+22:-(diff+22)]
    rotated_sudoku.append(x)
    #imshow(x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [115]:
# for idx,x in enumerate(rotated_sudoku):
#     print(idx)
#     clean = cv.cvtColor(x.copy(),cv.COLOR_GRAY2BGR)
#     img = cv.bitwise_not(x.copy())
#     thresh = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
#         cv.THRESH_BINARY,5,3)
#     contours,_ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#     mock_image = thresh.copy()
#     rect =  0
#     for cnt in contours:
#         x,y,w,h = cv.boundingRect(cnt)
#         epsilon = 0.1*cv.arcLength(cnt,True)
#         approx = cv.approxPolyDP(cnt,epsilon,True)
#         if len(approx) == 4:
#             cv.rectangle(clean,(x,y),(x+w,y+h),(0,255,0),2)
#             #saved = (x,y,x+w,y+h)
#     imshow(clean)

In [116]:
# sudoku_array = []
# vertical = []
# for img in rotated_sudoku:
#     edges = cv.Canny(img,50,100,apertureSize = 3)
#     img = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
#     lines = cv.HoughLines(edges,1,np.pi/180,200)
#     #print(lines[0])
#     per_image = []
#     if lines is not None:
#         for i in range(0, len(lines)):
#             rho = lines[i][0][0]
#             theta = lines[i][0][1]
#             a = math.cos(theta)
#             b = math.sin(theta)
#             #print(a,b)
#             x0 = a * rho
#             y0 = b * rho
#             pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
#             pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
#             if theta*57 >170 or theta*57 < 5:
#                 per_image.append([pt1,pt2])
#                 cv.line(img, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
#     vertical.append(per_image)
#     #imshow(img)

In [117]:
squares_per_image = []
for i in range(len(rotated_sudoku)):
    squares = []
    clean_image = rotated_sudoku[i].copy()
    mock_image = rotated_sudoku[i].copy()
    #mock_image = cv.GaussianBlur(mock_image, (3, 3),0)
    #mock_image = cv.bitwise_not(mock_image)
    mock_image = cv.adaptiveThreshold(mock_image,255,cv.ADAPTIVE_THRESH_MEAN_C,\
            cv.THRESH_BINARY,25,11)
    #imshow(mock_image)
    #kernel = np.ones((5,5), np.uint8)
    #mock_image  = cv.dilate(mock_image, kernel, iterations=1)
    contours,_ = cv.findContours(mock_image,cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    rect =  0
    clean_image = cv.cvtColor(clean_image,cv.COLOR_GRAY2BGR)
    for cnt in contours:
        x,y,w,h = cv.boundingRect(cnt)
        epsilon = 0.1*cv.arcLength(cnt,True)
        approx = cv.approxPolyDP(cnt,epsilon,True)
        perimeter = cv.arcLength(cnt,True)
        #print(len(approx))
        #print(x,y,w,h)
        if perimeter > 180 and perimeter < 300 and len(approx) == 4:
            #print(len(approx))
            cv.rectangle(clean_image,(x,y),(x+w,y+h),(0,255,0),2)
            #cv.drawContours(mock_image,[approx],0,(0,0,255),2)
            saved = (x,y,x+w,y+h)
            squares.append(saved)
    squares_per_image.append(squares)
    #imshow(clean_image)

In [118]:
count = 0 
for x in squares_per_image:
    if len(x) == 81:
        count+=1
    print(len(x))
print(count)

77
77
73
77
81
81
81
81
81
79
77
77
81
79
79
81
78
77
79
81
79
79
81
81
81
81
60
81
81
68
79
81
81
81
79
81
81
81
81
68
81
81
79
81
81
81
81
81
81
81
30


In [119]:
# imshow(rotated_sudoku[13])

In [120]:
def complete_squares(squares,img):
    squares =sorted(squares,key=lambda x: x[0])
    temp = sorted(squares,key=lambda x: x[1])
    a = [abs(temp[0][0]-temp[1][0]),abs(temp[0][1]-temp[1][1]),abs(temp[0][2]-temp[1][2]),abs(temp[0][3]-temp[1][3])]
    b = [abs(squares[0][0]-squares[1][0]),abs(squares[0][1]-squares[1][1]),abs(squares[0][2]-squares[1][2]),abs(squares[0][3]-squares[1][3])]
    if max(a)<=max(b):
        square_max = max(a)
    else:
        square_max = max(b)
    total = 0
    last = None
    start = 0
    fixed_sudoku = []
    square = []
    if len(squares) < 81:
        for i in squares:
            if last is None:
                total+=1
                last = i[0]
                start = i[1]
                square.append(i)
            elif len(square) == 9:
                fixed_sudoku.append(square)
                last = i[0]
                total=1
                start = i[1]
                square = []
                square.append(i)
            elif last+25 < i[0]:
                if total < 9:
                    timeout = 0
                    while len(square) !=9 and timeout != 100:
                        timeout+=1
                        square = sorted(square,key=lambda x: x[1])
                        min_y = min([x[1] for x in squares])
                        for j in range(len(square)):
                            if square[0][1]-10 <= min_y:
                                if j != len(square)-1 and len(square) < 9:
                                    
                                    if square[j+1][1]-square[j][1] >= square_max+16:
                                        elem = (square[j][0],square[j][3],square[j][2],square[j][3]+square_max-6)
                                        square.insert(j+1,elem)
                            else:
                                elem = (square[j][0],min_y,square[j][2],min_y+square_max)
                                square.insert(0,elem)
                        if len(square)== 9:
                            break
                        while len(square) <9 and img.shape[0]-square[-1][3]+10 >= square_max:
                            elem = (square[-1][0],square[-1][3],square[-1][2],square[-1][3]+square_max-6)
                            square.append(elem)
                    if len(square) == 9:
                        fixed_sudoku.append(square)
                else:
                    fixed_sudoku.append(square)
                last = i[0]
                total=1
                start = i[1]
                square = []
                square.append(i)
            else:
                last = i[0]
                square.append(i)
                total+=1
        if total !=9:
            timeout = 0
            while len(square) !=9 and timeout != 100:
                timeout+=1
                square = sorted(square,key=lambda x: x[1])
                min_y = min([x[1] for x in squares])
                for j in range(len(square)):
                    if square[0][1]-10 <= min_y:
                        if j != len(square)-1 and len(square) < 9:
                            if square[j+1][1]-square[j][1] >= square_max+16:
                                elem = (square[j][0],square[j][3],square[j][2],square[j][3]+square_max-6)
                                #print(elem)
                                square.insert(j+1,elem)
                    else:
                        elem = (square[j][0],min_y,square[j][2],min_y+square_max)
                        square.insert(0,elem)
                if len(square)== 9:
                    break
                while len(square) <9 and img.shape[0]-square[-1][3] >= square_max:
                    elem = (square[-1][0],square[-1][3],square[-1][2],square[-1][3]+square_max-6)
                    square.append(elem)
                #print(sorted(square,key=lambda x: x[1]))
            if len(square) == 9:
                fixed_sudoku.append(square)
            else:
                pass
        else:
            fixed_sudoku.append(square)
        return fixed_sudoku
    return squares


In [121]:
complete_squares(squares_per_image[11],rotated_sudoku[11])

[[(19, 90, 80, 155),
  (21, 226, 80, 288),
  (21, 157, 80, 220),
  (21, 25, 80, 90),
  (22, 290, 81, 355),
  (23, 424, 82, 488),
  (23, 357, 81, 420),
  (25, 487, 83, 553),
  (26, 552, 84, 619)],
 [(82, 225, 148, 288),
  (82, 156, 147, 219),
  (82, 89, 147, 154),
  (82, 22, 146, 87),
  (83, 357, 148, 421),
  (83, 290, 148, 356),
  (84, 426, 148, 489),
  (85, 490, 149, 556),
  (86, 556, 150, 623)],
 [(218, 89, 281, 155),
  (218, 23, 281, 88),
  (219, 490, 282, 556),
  (219, 425, 282, 489),
  (219, 356, 283, 420),
  (219, 289, 282, 355),
  (219, 225, 282, 288),
  (219, 157, 282, 219),
  (220, 557, 283, 623)],
 [(283, 90, 348, 155),
  (283, 23, 348, 88),
  (284, 424, 349, 488),
  (284, 289, 348, 353),
  (284, 225, 348, 287),
  (284, 157, 348, 219),
  (285, 555, 349, 621),
  (285, 489, 349, 554),
  (285, 355, 349, 418)],
 [(350, 355, 414, 419),
  (350, 289, 413, 353),
  (350, 224, 412, 287),
  (350, 156, 413, 220),
  (350, 89, 413, 155),
  (350, 22, 413, 88),
  (351, 424, 414, 487),
  (352

In [122]:
template = '''
oooxxxooo
ooxoooxoo
xooooooox
oxoooooxo
oxoxxxoxo
oxoooooxo
xooooooox
ooxoooxoo
oooxxxooo'''
template = template.replace("x",'o').split("\n")[1:]
template = [list(x) for x in template]
print(template)

[['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'], ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']]


In [123]:
# img = rotated_sudoku[6].copy()

# item = complete_squares(squares_per_image[6],img)
# if len(item) < 81 :
#     new_list = []
#     for x in item:
#         for s in x:
#             new_list.append(s)
#     block = sorted(new_list,key=lambda x: (x[0],x[1]))
#     value = np.abs((np.array(block[0]) - np.array(block[1]))).max()-1
#     min_start = sorted(new_list,key=lambda x: (x[0]))[0][0]
# else:
#     block = sorted(item,key=lambda x: (x[0],x[1]))
#     value = np.abs((np.array(block[0]) - np.array(block[1]))).max()-1
#     min_start = sorted(item,key=lambda x: (x[0]))[0][0]
#     min_up = sorted(item,key=lambda x: (x[1]))[0][1]
#     new_list = item
# print(value)
# print(min_up)
# print(np.abs((np.array(block[0]) - np.array(block[1]))).max())
# values = []
# picture = cv.cvtColor(img.copy(),cv.COLOR_GRAY2BGR)
# img = cv.bitwise_not(img)
# for x,y,w,h in new_list:
#     cv.rectangle(picture,(x,y),(w,h),(0,255,0),2)
#     uniq = np.unique(np.digitize(img[y+6:h-6,x+6:w-6],np.array([55])), return_counts=True)
#     print(x,y)
#     if x < value and x < value-10:
#         i=0
#     elif x > value-10 and x < value:
#         i = x//value +1
#     else:
#         i = x//value
#     if y < value and y < value-10:
#         j=0
#     elif y > value-10 and y < value:
#         j = y//value +1
#     else:
#         j = y//value
#     print(i,j)
#     print(uniq)
#     if len(uniq[1])== 2 and uniq[1][1] > 100:
#         #print(j,i)
#         values.append((j,i))
#     imshow(picture)
    
#     #imshow(view[y+8:h-8,x+8:w-8])
        
# print(values)
# print(len(values))

In [124]:
# file = open('../train/classic/'+str(6+1)+'_gt.txt','r')
# entity = []
# for x in file.readlines():
#     entity.append(list(x.replace("\n",'')))
# for x in values:
#     i,j = x[0],x[1]
#     if j > 8 :
#         j=8
#     if i < 0:
#         i=0
#     if i >8:
#         i=0
#     print(i,j)
#     print(entity[i][j])

In [125]:
import numpy as np
import copy
per_image = []
for idx in range(1,51):
    print('picture idx ',idx)
    submision = copy.deepcopy(template)
    file = open('../train/classic/'+str(idx)+'_gt.txt','r')
    entity = []
    for x in file.readlines():
        entity.append(list(x.replace("\n",'')))
    count = 0 
    img = rotated_sudoku[idx-1].copy()
    img = cv.bitwise_not(img)
    print('Len - ',len(squares_per_image[idx-1]))
    #print(block)
    values = []
    if len(squares_per_image[idx-1]) == 81:
        block = sorted(squares_per_image[idx-1],key=lambda x: (x[0],x[1]))
        print(np.abs((np.array(block[0]) - np.array(block[1]))))
        block = np.abs((np.array(block[0]) - np.array(block[1]))).max()-1
        for s,y,w,h in sorted(squares_per_image[idx-1],key=lambda x: (x[0],x[1])):
            uniq = np.unique(np.digitize(img[y+8:h-8,s+8:w-8],np.array([55])), return_counts=True)
            if s < block and s < block-10:
                i=0
            elif s > block-10 and s < block:
                i = s//block +1
            else:
                i = s//block
            if y < block and y < block-10:
                j=0
            elif y > block-10 and y < block:
                j = y//block +1
            else:
                j = y//block
            #print(i,j)
            if len(uniq[1])== 2 and uniq[1][1] > 100:
                #print(j,i)
                values.append((j,i))
                #submision[j][i] = 'x'
                #imshow(img[y+8:h-8,s+8:w-8],)
            #print(uniq)
            count+=1
    else:
        x = complete_squares(squares_per_image[idx-1],img)
        new_list = []
        for item in x:
            for s in item:
                new_list.append(s)
        new_list = sorted(new_list,key=lambda x: (x[0],x[1]))
        block = sorted(new_list,key=lambda x: (x[0],x[1]))
        print(np.abs((np.array(block[0]) - np.array(block[1]))))
        block = np.abs((np.array(block[0]) - np.array(block[1]))).max()-1
        for s,y,w,h in new_list:
            i,j = s//block ,y//block
            
            uniq = np.unique(np.digitize(img[y+8:h-8,s+8:w-8],np.array([55])), return_counts=True)
            if len(uniq[1])== 2 and uniq[1][1] > 100:
                #submision[j][i] = 'x'
                values.append((j,i))
                #imshow(img[y+8:h-8,s+8:w-8],)
                #print(i,j)
            count+=1
    per_image.append(values)

picture idx  1
Len -  77
[ 0 65  1 66]
picture idx  2
Len -  77
[ 0 59  0 53]
picture idx  3
Len -  73
[ 0 60  0 59]
picture idx  4
Len -  77
[ 0 66  1 67]
picture idx  5
Len -  81
[ 1 56  1 58]
picture idx  6
Len -  81
[ 0 67  1 66]
picture idx  7
Len -  81
[ 0 64  0 61]
picture idx  8
Len -  81
[ 0 63  0 63]
picture idx  9
Len -  81
[ 0 57  0 55]
picture idx  10
Len -  79
[ 0 55  1 55]
picture idx  11
Len -  77
[ 0 67  1 67]
picture idx  12
Len -  77
[ 2 65  0 65]
picture idx  13
Len -  81
[ 0 65  0 66]
picture idx  14
Len -  79
[ 0 60  0 59]
picture idx  15
Len -  79
[ 0 61  1 61]
picture idx  16
Len -  81
[ 0 68  0 67]
picture idx  17
Len -  78
[ 0 60  0 60]
picture idx  18
Len -  77
[ 0 62  0 56]
picture idx  19
Len -  79
[ 0 56  0 50]
picture idx  20
Len -  81
[ 0 58  1 61]
picture idx  21
Len -  79
[ 0 62  0 56]
picture idx  22
Len -  79
[ 0 63  1 63]
picture idx  23
Len -  81
[ 0 66  0 68]
picture idx  24
Len -  81
[ 0 69  0 70]
picture idx  25
Len -  81
[ 1 68  0 68]
picture i

In [126]:
len(squares_per_image[11])

77

In [127]:
# imshow(rotated_sudoku[11])

In [128]:
len(per_image)

50

In [129]:
total_wrong = 0
count = 0 
for idx,x in enumerate(per_image):
    #print(idx)
    submision = copy.deepcopy(template)
    first = np.array([y[0] for y in x])
    second = np.array([y[1] for y in x])
    if min(first) < 0:
        first+=abs(min(first))
    if min(second) < 0:
        first+=abs(min(second))
    if max(first) > 8:
        if 0 in first:
            first[first > 8] = 8
        else:
            #print(first)
            first-=(max(first)-8)
            #print(first)
    if max(second) > 8:
        if 0 in second:
            second[second > 8] = 8
        else:
            #print(second)
            second-= (max(second)-8)
            #print(second)
    file = open('../train/classic/'+str(idx+1)+'_gt.txt','r')
    entity = []
    for x in file.readlines():
        entity.append(list(x.replace("\n",'')))

    for x,y in zip(first,second):
        #print(entity[x][y])
        submision[x][y] = 'x'
    if entity == submision:
        count+=1
    else:
        amount_right = 0
        for i in range(9):
            for j in range(9):
                if submision[i][j] == entity[i][j]:
                    amount_right+=1
        #print(amount_right,"/",81)
        total_wrong+=amount_right
        print(total_wrong)
print(count)
print(total_wrong)

80
153
231
305
355
434
514
562
615
695
40
695


In [130]:
entity

[['o', 'o', 'o', 'x', 'o', 'o', 'x', 'o', 'o'],
 ['x', 'o', 'x', 'o', 'x', 'o', 'x', 'o', 'o'],
 ['o', 'x', 'o', 'o', 'o', 'x', 'o', 'o', 'x'],
 ['x', 'o', 'o', 'o', 'o', 'o', 'x', 'o', 'x'],
 ['o', 'x', 'o', 'o', 'x', 'o', 'o', 'x', 'o'],
 ['x', 'o', 'x', 'o', 'o', 'o', 'o', 'o', 'x'],
 ['x', 'o', 'o', 'x', 'o', 'o', 'o', 'x', 'o'],
 ['o', 'o', 'x', 'o', 'x', 'o', 'x', 'o', 'x'],
 ['o', 'o', 'x', 'o', 'o', 'x', 'o', 'o', 'o']]

In [131]:
complete_squares(squares_per_image[48],img)

[(16, 295, 69, 347),
 (16, 240, 69, 294),
 (16, 186, 69, 239),
 (16, 129, 69, 183),
 (16, 74, 68, 128),
 (16, 19, 67, 73),
 (17, 351, 70, 402),
 (18, 403, 71, 455),
 (19, 456, 72, 507),
 (69, 19, 124, 73),
 (70, 187, 125, 239),
 (70, 130, 125, 183),
 (70, 74, 124, 129),
 (71, 350, 126, 402),
 (71, 295, 126, 347),
 (71, 240, 125, 294),
 (72, 403, 126, 455),
 (73, 455, 127, 506),
 (125, 19, 179, 73),
 (126, 130, 180, 183),
 (126, 74, 180, 129),
 (127, 401, 180, 454),
 (127, 349, 180, 401),
 (127, 294, 180, 346),
 (127, 240, 180, 293),
 (127, 187, 180, 239),
 (128, 454, 180, 504),
 (183, 19, 237, 74),
 (184, 452, 236, 503),
 (184, 400, 236, 452),
 (184, 348, 236, 400),
 (184, 293, 236, 345),
 (184, 240, 236, 292),
 (184, 187, 236, 238),
 (184, 131, 237, 183),
 (184, 75, 237, 130),
 (237, 451, 289, 501),
 (237, 399, 289, 451),
 (237, 347, 289, 398),
 (238, 293, 289, 344),
 (238, 239, 290, 292),
 (238, 187, 290, 238),
 (238, 131, 290, 183),
 (238, 76, 291, 130),
 (238, 20, 292, 75),
 (290, 

In [132]:
lines_coord = []
for x in vertical:
    per_img = []
    for y in x:
        per_img.append((y[0][0]+y[1][0])//2)
    lines_coord.append(sorted(per_img))
#     print(x[0][0])
#     print(x[0][1])
#     print(rotated_sudoku[0].shape)
#     #print(sorted(x, key = lambda s: s[0][0][0]))
#     cv.line(rotated_sudoku[0], x[0][0], x[0][1], (0,0,255), 3, cv.LINE_AA)
#     imshow(rotated_sudoku[0][-1007:,x[0][0][0]:x[0][1][0]])

NameError: name 'vertical' is not defined